In [34]:
import pandas as pd
import nltk
import sklearn

In [2]:
movies=pd.read_csv("my-projects-in-predictive-modelling-and-NLP/modelling-folder/movie-recomendation-system/tmdb_5000_movies.csv")
credits=pd.read_csv("my-projects-in-predictive-modelling-and-NLP/modelling-folder/movie-recomendation-system/tmdb_5000_credits.csv")

In [35]:
print("pandas",pd.__version__)
print("nltk",nltk.__version__)
print("sklearn",sklearn.__version__)

pandas 2.2.3
nltk 3.9.1
sklearn 1.6.1


In [3]:
movies.shape

(4803, 20)

In [4]:
credits.shape

(4803, 4)

In [5]:
movies.rename(columns={'id':"movie_id"},inplace=True)

data=movies.merge(credits,on='movie_id')

In [6]:
data.columns

data.rename(columns={'title_x':'title'},inplace=True)

data.columns

data.sample(1)

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,title_y,cast,crew
718,60000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",NaN,8914,"[{""id"": 2988, ""name"": ""shark attack""}, {""id"": ...",en,Deep Blue Sea,On a remote former submarine refueling facilit...,22.991269,"[{""name"": ""Village Roadshow Pictures"", ""id"": 7...",...,105.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Bigger. Smarter. Faster. Meaner.,Deep Blue Sea,5.6,604,Deep Blue Sea,"[{""cast_id"": 14, ""character"": ""Carter Blake"", ...","[{""credit_id"": ""52fe44c7c3a36847f80a9971"", ""de..."


In [7]:
columns_to_remove=['budget','homepage','original_language','original_title','popularity','production_countries','release_date','revenue','runtime','spoken_languages','status','vote_average','vote_count','title_y']

data=data.drop(columns=columns_to_remove,axis=1)



In [8]:
data.head(1)

,genres,movie_id,keywords,overview,production_companies,tagline,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Enter the World of Pandora.,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
from ast import literal_eval

In [10]:
def extract_names(data):
    names=[]
    for items in literal_eval(data):
        names.append(items['name'])
    return names

In [12]:
data['genres']=data['genres'].apply(extract_names)

In [14]:
data['keywords']=data['keywords'].apply(extract_names)

In [16]:
data['production_companies']=data['production_companies'].apply(extract_names)

In [17]:
def extract_top_names(data):
    names=[]
    counter=0
    for items in literal_eval(data):
        if counter<3:
            names.append(items['name'])
            counter+=1
    return names

In [19]:
data['cast']=data['cast'].apply(extract_top_names)

In [20]:
def fetch_director(data):
    name=[]
    for item in literal_eval(data):
        if item['job']=='Director':
            name.append(item['name'])
    return name

In [22]:
data['crew']=data['crew'].apply(fetch_director)

In [24]:
import string

In [26]:
data['overview']=data['overview'].str.translate(str.maketrans('', '',string.punctuation))

In [36]:
from nltk.tokenize import word_tokenize

In [37]:
def tokenizer(data):
    for item in data:
        return word_tokenize(item)

In [38]:
data.isnull().sum()

genres                    0
movie_id                  0
keywords                  0
overview                  3
production_companies      0
tagline                 844
title                     0
cast                      0
crew                      0
dtype: int64

In [39]:
data.drop(columns='tagline',axis=1,inplace=True)

In [40]:
data=data[~data['overview'].isnull()]

In [42]:
data['overview']=data['overview'].apply(word_tokenize)

In [43]:
def remove_spaces(data):
    names=[]
    for item in data:
        names.append(item.replace(" ",""))
    return names


In [44]:
data['cast']=data['cast'].apply(remove_spaces)

In [45]:
data['crew']=data['crew'].apply(remove_spaces)

In [46]:
data['production_companies']=data['production_companies'].apply(remove_spaces)

In [ ]:
data['tags']=data['genres']+data['keywords']+data['overview']+data['production_companies']+data['cast']+data['crew']

In [48]:
def to_lower(data):
    lst=[]
    for item in data:
        lst.append(item.lower())
    return lst

In [49]:
data.head()

,genres,movie_id,keywords,overview,production_companies,title,cast,crew,tags
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...","[In, the, 22nd, century, a, paraplegic, Marine...","[IngeniousFilmPartners, TwentiethCenturyFoxFil...",Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, Science Fiction, ..."
1,"[Adventure, Fantasy, Action]",285,"[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa, long, believed, to, be, de...","[WaltDisneyPictures, JerryBruckheimerFilms, Se...",Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, ocean, drug abuse..."
2,"[Action, Adventure, Crime]",206647,"[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond, ’, s, past, ...","[ColumbiaPictures, Danjaq, B24]",Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime, spy, based on novel..."
3,"[Action, Crime, Drama, Thriller]",49026,"[dc comics, crime fighter, terrorist, secret i...","[Following, the, death, of, District, Attorney...","[LegendaryPictures, WarnerBros., DCEntertainme...",The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,"[Action, Adventure, Science Fiction]",49529,"[based on novel, mars, medallion, space travel...","[John, Carter, is, a, warweary, former, milita...",[WaltDisneyPictures],John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, Science Fiction, based on ..."


In [50]:
data.drop(columns=['genres','keywords','overview','production_companies','cast','crew'],axis=1,inplace=True)

In [51]:
data['tags']=data['tags'].apply(remove_spaces)

In [52]:
data['tags']=data['tags'].apply(to_lower)

In [53]:
data['tags']=data['tags'].apply(lambda x:" ".join(x))

In [54]:
data.head()

,movie_id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction cultur...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...
2,206647,Spectre,action adventure crime spy basedonnovel secret...
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...
4,49529,John Carter,action adventure sciencefiction basedonnovel m...


In [55]:
data['tags'][0]

'action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d in the 22nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization ingeniousfilmpartners twentiethcenturyfoxfilmcorporation duneentertainment lightstormentertainment samworthington zoesaldana sigourneyweaver jamescameron'

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [57]:
vectorizer=CountVectorizer(max_features=5000,stop_words='english')

In [58]:
vectors=vectorizer.fit_transform(data['tags']).toarray()

In [59]:
from nltk.stem.porter import PorterStemmer

In [60]:
stemmer=PorterStemmer()

In [61]:
def stem_text(data):
    l=[]
    for element in data.split():
        l.append(stemmer.stem(element))
    return " ".join(l)

In [62]:
data['tags']=data['tags'].apply(stem_text)

In [63]:
vectors=vectorizer.fit_transform(data['tags']).toarray()

In [64]:
vectorizer.get_feature_names_out()

array(['007', '10', '100', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [65]:
data['tags'][0]

'action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d in the 22nd centuri a parapleg marin is dispatch to the moon pandora on a uniqu mission but becom torn between follow order and protect an alien civil ingeniousfilmpartn twentiethcenturyfoxfilmcorpor duneentertain lightstormentertain samworthington zoesaldana sigourneyweav jamescameron'

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
vectors.shape

(4800, 5000)

In [68]:
similarity=cosine_similarity(vectors)

In [25]:
data.sample(1)

,movie_id,title,tags
4088,80,Before Sunset,drama romanc pari journalist dialogu talk soul...


In [69]:
data[data['title']=='Avatar'].index[0]

np.int64(0)

In [70]:
def recommend(movie):
    movie_index=data[data['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for movie in movie_list:
        print(data.iloc[movie[0]].title)

In [72]:
recommend('Batman')

Batman & Robin
Batman Begins
Batman Returns
The R.M.
Batman Forever


In [73]:
import pickle

In [75]:
pickle.dump(data,open('movies.pkl','wb'))

In [76]:
pickle.dump(similarity,open('similarity.pkl','wb'))